**Importing Modules**

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML

**Distance Function** <br>
The function takes the numpy array of `data`, `centers` and `dist` as paramaters. The `data` array contains the scaled credit card data while the `centers` array stores the indices of chosen centers. The `dist` array stores the minimum distance of each point from its nearest cluster center and gets updated on each iteration. The function returns the cost of clustering till the current iteration and the latest chosen center index.

In [ ]:
def distance(data,centers,dist):
  size = len(data)
  index = len(centers) - 1
  cost = -1

  for i in range(size):
    dist[i] = min(dist[i],np.linalg.norm(data[i] - data[centers[index]]))
    if(dist[i] > cost):
      cost = dist[i]
      Ci = i
      
  return Ci,cost

**Greedy K Center** <br>
The function takes the numpy array of `cc_data`, `centers` and `k` as paramaters. The loop runs till k centers are chosen by calling the `distance` function for each new center added. An array `dist` is created to store the minimum distance of each point from its nearest cluster center. The `distance` function works on the invariant that the minimum distance of each point to its nearest cluster center either remains same or decreases when a new center is chosen. So in each iteration distances of points to the newly added center are only calculated. 

In [ ]:
def kcenter(k,cc_data,centers):
  dist = [math.inf]*len(cc_data)

  for i in range(k):
    Ci,cost = distance(cc_data,centers,dist)
    centers.append(Ci)

  return centers,cost

**Optimal K Center** <br>
The function takes the numpy array of `opt_data` and `k` as parameters. For `k=2` and `k=4` we loop over all possible choices of centers and find the choice of centers with least clustering cost. in each iteration of the innermost loop, the distance of each point to the choice of `k` centers is calculated. This is accomplished by calling adding the appropriate center indices to `centers` array and then calling `distance` function on it till `k` centers are chose. If the clustering cost is less than the previously computed costs, the cost and choice of centers are updated.  

In [ ]:
def optimal(k,opt_data):
  size = len(opt_data)
  min_cost = math.inf
  centers = [0]*k
  result = [0]*k

  if(k==2):
    for i in range(size-1):
      for j in range(i+1,size):
        dist = [math.inf]*size
        centers[0] = i
        Ci,cost = distance(opt_data,centers,dist)
        centers[1] = j
        Ci,cost = distance(opt_data,centers,dist)

        if(cost < min_cost):
          min_cost = cost
          result = centers

  elif(k==4):
    for i in range(size-3):
      for j in range(i+1,size-2):
        for k in range(j+1,size-1):
          for l in range(k+1,size):
            dist = [math.inf]*size
            centers[0] = i
            Ci,cost = distance(opt_data,centers,dist)
            centers[1] = j
            Ci,cost = distance(opt_data,centers,dist)
            centers[2] = k
            Ci,cost = distance(opt_data,centers,dist)
            centers[3] = l
            Ci,cost = distance(opt_data,centers,dist)

            if(cost < min_cost):
              min_cost = cost
              result = centers

  return result,min_cost

**K Center** <br>
We first import the data set and scale it using `MinMaxScaler`. The index is set to `CUST_ID`. The `scaler` transforms the dataframe to a numpy array and the first center is chosen randomly. Then the respective calls to greedy k center and optimal k center functions are made and the costs for each clustering are calculated. 

In [ ]:
path = "https://raw.githubusercontent.com/kushagra-18110091/http_test/main/CC%20GENERAL.csv"  
scaler = MinMaxScaler()

cc_data = pd.read_csv(path).fillna(0)
cc_data = cc_data.set_index('CUST_ID')
cc_data = scaler.fit_transform(cc_data)

length = (int)(len(cc_data)/200)

k = [2,4,10]
values_cc = []
values_opt = []

for i in k:
  centers = []
  opt_data = cc_data[np.random.choice(cc_data.shape[0], length, replace=False), :]
  centers,opt_cost = optimal(i,opt_data)
  
  centers = []
  centers.append(np.random.choice(range(len(cc_data))))
  i = i - 1
  centers,greedy_cc = kcenter(i,cc_data,centers)
  i = i + 1

  centers = []
  centers.append(np.random.choice(range(len(opt_data))))
  i = i - 1
  centers,greedy_opt = kcenter(i,opt_data,centers)
  
  if(i+1 == 10):
    values_cc.append([i+1,greedy_cc,'-','-'])
    values_opt.append([i+1,greedy_opt,'-','-'])
  else:
    values_cc.append([i+1,greedy_cc,opt_cost,greedy_cc/opt_cost])
    values_opt.append([i+1,greedy_opt,opt_cost,greedy_opt/opt_cost])

print("Greedy with Original Data and Optimal with 1/200th of original data")
result_cc = pd.DataFrame(values_cc, columns=["K","Greedy","Optimal","Approxiation Ratio"])
display(HTML(result_cc.to_html()))

print()
print("Greedy and Optimal with 1/200th of original data")
result_opt = pd.DataFrame(values_opt, columns=["K","Greedy","Optimal","Approxiation Ratio"])
display(HTML(result_opt.to_html()))


Greedy with Original Data and Optimal with 1/200th of original data


,K,Greedy,Optimal,Approxiation Ratio
0,2,2.355714,1.20721,1.95137
1,4,1.860964,1.12275,1.6575
2,10,1.405193,-,-



Greedy and Optimal with 1/200th of original data


,K,Greedy,Optimal,Approxiation Ratio
0,2,1.688159,1.20721,1.3984
1,4,1.247570,1.12275,1.11117
2,10,0.718392,-,-


The Approximation Ratio is less than 2 when we take the reduced data set for both greedy and optimal solutions. While the ratio comes close to 2 when we take original data set for greedy and reduced data set for optimal. A proper analysis can only be made when the data sets for both the solutions are same. Thus the second scenario gives more accurate approximation ratio. 